In [2]:
import os

import cv2 as cv
import numpy as np
from ultralytics import YOLO

model_path = os.path.join(os.getcwd(), "YOLO_model", "weights", "best.pt")
model = YOLO(model_path)

In [3]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [4]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape
    print(labels)
    print(res.boxes.conf)
    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [11]:
img = cv.imread("scr3.png")[:, :, :3]
img = cv.resize(img, (640, 640))
res = find_pieces(img)
from analysis import get_fen

print(get_fen(res, "w", "w"))
print(res)


0: 640x640 1 black-king, 6 black-pawns, 1 black-queen, 1 black-rook, 2 white-bishops, 1 white-king, 6 white-pawns, 1 white-queen, 290.4ms
Speed: 3.9ms preprocess, 290.4ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)


['black-queen', 'white-king', 'white-queen', 'black-pawn', 'black-rook', 'black-king', 'black-pawn', 'white-pawn', 'black-pawn', 'white-pawn', 'white-bishop', 'white-bishop', 'white-pawn', 'black-pawn', 'white-pawn', 'white-pawn', 'white-pawn', 'black-pawn', 'black-pawn']
tensor([0.9922, 0.9725, 0.9338, 0.9049, 0.8876, 0.8849, 0.8363, 0.8297, 0.8137, 0.8020, 0.7915, 0.7897, 0.7894, 0.7527, 0.7215, 0.7195, 0.6860, 0.6493, 0.6231])
5rk1/1pq3pp/p7/3p1p2/Q2B4/2P4B/PP3PPP/6K1 w
[['' '' '' '' '' 'r' 'k' '']
 ['' 'p' 'q' '' '' '' 'p' 'p']
 ['p' '' '' '' '' '' '' '']
 ['' '' '' 'p' '' 'p' '' '']
 ['Q' '' '' 'B' '' '' '' '']
 ['' '' 'P' '' '' '' '' 'B']
 ['P' 'P' '' '' '' 'P' 'P' 'P']
 ['' '' '' '' '' '' 'K' '']]
